In [1]:
from glob import glob
import json
import os
import pandas as pd

import pypowsybl as pp

from IPython.core.display import display,SVG,HTML

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import plotly.graph_objects as go

from dash_canvas import DashCanvas

from jupyter_dash import JupyterDash

In [2]:
# write width and height attributes in SDL's SVG
def fixSvg(svgs, width=700, height=700):
    SVG_HEADER1 = '<svg xmlns=\"http://www.w3.org/2000/svg\">'
    SVG_HEADER2 = '<svg xmlns=\"http://www.w3.org/2000/svg\" width=\"{w:}px\" height=\"{h:}px\" >'
    return svgs.replace(SVG_HEADER1, SVG_HEADER2.format(w=width, h=height))


In [3]:
# save the SVG string to FS
def saveSvg(svg_string, filename):
    svgFile = open(filename, "wt")
    n = svgFile.write(svg_string)
    svgFile.close() 

In [4]:
# get a network
n = pp.network.create_four_substations_node_breaker_network()

subId = 'S1'
vlid = 'S1VL1'

In [5]:
# write the SVGs to the asset folders (directory served by dash, by default)
vlSvg1 = fixSvg(str(n.get_single_line_diagram(vlid)), width=800, height=800);
saveSvg(vlSvg1, 'assets/vl1_1.svg')

subSvg2 = fixSvg(str(n.get_single_line_diagram(subId)), width=1700, height=800);
saveSvg(subSvg2, 'assets/sub2_1.svg')

filenames = sorted(glob(os.path.join('assets', '*.svg')))


In [6]:
#show SVG - plotly and Jupyter-dash

app = JupyterDash(__name__)

app.layout = html.Div([
    html.Label([ 
        "Networks SLDs", 
        dcc.Dropdown(
            id='filenames-dropdown', clearable=False,
            value=filenames[0], options=[{'label': c, 'value': c}
                                         for c in filenames])
    ]),
    dcc.Graph(id="fig-sdl", 
           config = dict({'displayModeBar': True, 'scrollZoom': True, 'displaylogo': False, 
                          'modeBarButtonsToRemove': ['toImage', 'zoom', 'autoscale', 'resetscale']}))
])

@app.callback(
    Output('fig-sdl', 'figure'),
    [Input("filenames-dropdown", "value")]
)
def update_figure(filename):
    fig = go.Figure()
    fig.add_layout_image(
        dict( source=filename, xref="x",yref="y", x=0, y=0, sizex=6, sizey=6,
        xanchor="left", yanchor="bottom", opacity=1, layer="above"
        )
    )
    
    fig.update_xaxes(showgrid=False, zeroline=False, rangemode="tozero", showticklabels=False)
    fig.update_yaxes(showgrid=False, zeroline=False, rangemode="tozero", showticklabels=False)
    fig.update_layout(dragmode="pan",  autosize=False, width=950, height=950, 
                     margin={'l': 0, 'b': 0, 't': 0, 'r': 0})
    return fig;

app.run_server(mode='inline', height='1200', port=8051)